In [40]:
import numpy as np
from sklearn.tree             import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection  import train_test_split,GridSearchCV,ShuffleSplit
from sklearn.datasets         import make_classification
from graphviz                 import Source
from sklearn.metrics          import *

In [19]:
data = make_classification(n_samples=10_000,n_classes=2,n_features=10)

In [22]:
X = data[0]
y = data[1]

In [24]:
xtrain ,xtest ,ytrain ,ytest = train_test_split(X,y,test_size=0.2,random_state=42)

In [26]:
tree = DecisionTreeClassifier()
tree.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [35]:
tree.get_depth(),tree.tree_.n_leaves,tree.tree_.node_count

(22, 277, 553)

In [38]:
pred = tree.predict(xtest)

print(classification_report(ytest,pred))
print(confusion_matrix(ytest,pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       989
           1       0.95      0.95      0.95      1011

    accuracy                           0.95      2000
   macro avg       0.95      0.95      0.95      2000
weighted avg       0.95      0.95      0.95      2000

[[939  50]
 [ 48 963]]


In [43]:
params = dict(
    max_depth = np.arange(3,20),
    min_samples_split = [100,150,200],
)
gridTree = GridSearchCV(estimator=DecisionTreeClassifier(max_features=5),param_grid=params,scoring='accuracy')
gridTree.fit(xtrain,ytrain)

print(gridTree.best_estimator_)
print(gridTree.best_score_)

DecisionTreeClassifier(max_depth=17, max_features=5, min_samples_split=100)
0.9595


In [44]:
tree = DecisionTreeClassifier(max_depth=17, max_features=5, min_samples_split=100)
tree.fit(xtrain,ytrain)
pred = tree.predict(xtest)

print(classification_report(ytest,pred))
print(confusion_matrix(ytest,pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       989
           1       0.97      0.95      0.96      1011

    accuracy                           0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000

[[960  29]
 [ 46 965]]


In [32]:
export_graphviz(decision_tree=tree,out_file='img/tree.dot')
with open('img/tree.dot','r') as file:
    dot = file.read()
#Source(dot)

In [67]:
from sklearn.base import ClassifierMixin
from statistics   import mode
class RandomFlorest(ClassifierMixin):
    treinamentos = []
    def __init__(self,Ninstance=1000,splits=10,CVtree=DecisionTreeClassifier()) -> None:
        self.Ninstance = splits
        self.splits    = Ninstance
        self.Tree      = CVtree


    def fit(self,X,y):
        self.treinamentos = [self.Tree for i in range(self.splits)] 
        for enum,(train,test) in enumerate(ShuffleSplit(n_splits=self.splits,train_size=self.Ninstance).split(X,y)):
        
            xtrainf , xtestf = X[train,:],X[test,:]
            ytrainf =  y[train]
            self.treinamentos[enum].fit(xtrainf,ytrainf)

    def predict(self,X):
        dados = []
        if np.size(X,axis=0) > 1:
            for test in X:
                moda = []
                for trees in self.treinamentos:
                    moda.append(trees.predict(np.array([test]))[0])
                dados.append(mode(moda))
            return dados
        


        
random = RandomFlorest(1000,10,tree)
random.fit(xtrain,ytrain)

pred = random.predict(xtest)

In [68]:
accuracy_score(ytest,pred)

0.5055